# Training file Hotel Id assignment #
*more info here*

# Imports, Setup, etc. #

In [ ]:
# Imports:
! pip install timm --no-index --find-links=file:///kaggle/input/timm-package/timm

In [ ]:
import numpy as np # linear algebra
import math as m
from datetime import datetime
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torchvision
import random
import numpy.random as rnd
import os
import matplotlib.pyplot as plt
from PIL import Image as pil_image
import albumentations.pytorch as APT
import albumentations as A
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import timm


# Don't know what this does, so I'll just keep it there just in case...
#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        pass
        #print(os.path.join(dirname, filename))

# File & Folder names (not all used atm):        
PROJECT_FOLDER = "../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/"
DATA_FOLDER = "../input/hotelid-2022-train-images-256x256/" # I added this one but we do not use it currently... (includes all pictures shuffled and padded.)
IMAGE_FOLDER = DATA_FOLDER + "images/"
OUTPUT_FOLDER = ""

# NMBR_HOTELS = 3116
NMBR_HOTELS = 100
MAX_WINDOWS_FILENAME_CHAR_LENGTH = 260 
NMBR_EMBEDDINGS = 20


# Image Processing
##### Includes creating datasets & Loaders, and data transformations

### Dataset class v1
##### Written by Nicolay, currently unused

In [ ]:
class HotelDataset_Nicolay(Dataset):
    """Dataset used for testing and validation set of hotel images."""
    
    def __init__(self, DataSet=None,root_dir="../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/", data_path="train_images/", test_data_path="test_images/", validation_percent=0.3, min_images=1, max_images=None, max_entries=None, transform=None):
        
        # Setting variables:
        
        MAX_WINDOWS_FILENAME_CHAR_LENGTH = 260
        self.transform = transform
        # Directory paths:
        self.root_dir = root_dir
        self.data_path = data_path
        self.test_data_path = test_data_path
        self.full_data_path = os.path.join(root_dir, data_path)
        self.full_test_data_path = os.path.join(root_dir, test_data_path)
        
        # Directory lists
        dirs = os.listdir(self.full_data_path)
        tests_dirs = os.listdir(self.full_test_data_path)
        
        # Initialise as empty dataset:
        if DataSet != None:
            self.hotel_data = DataSet[0]
            self.hotel_test_data = DataSet[0]
            self.total_files = DataSet[1]
            return

        # Scanning for 'usefull' directories:
        
        self.num_labels = len(dirs)
        self.total_files = 0
        for directory in dirs:
            files=os.listdir(os.path.join(self.full_data_path,directory))
            files_number = len(files)
            # Remove file if it doesn't have required # of images (note: why is there a maximum, can't we just take less out of these dirs?)
            if (max_images != None and files_number > max_images) or (files_number < min_images):
                dirs.remove(directory)
                continue
            self.total_files += files_number
            # Stop if max #pictures has been reached
            if max_entries != None and max_entries >= self.total_files:
                dirs = dirs[:max_entries]
                break
        
        # Load Test & Validation data :
        
        self.hotel_data = np.chararray([self.total_files,2], itemsize=MAX_WINDOWS_FILENAME_CHAR_LENGTH)
        iterator = 0
        for directory in dirs:
            files=os.listdir(os.path.join(self.full_data_path,directory))
            for f in files:
                if not iterator < self.total_files:
                    break
                self.hotel_data[iterator] = [f, directory]
                iterator = iterator + 1
        
        # Defide into validation and testing sets:
        
        # Validation set:
        self.validation_files = self.total_files * validation_percent
        self.validation_files = round(self.validation_files)
        total_files_copy = self.total_files - self.validation_files
        self.validation_data = np.chararray([self.validation_files,2], itemsize=MAX_WINDOWS_FILENAME_CHAR_LENGTH)
        hotel_data_copy = np.chararray([total_files_copy,2], itemsize=MAX_WINDOWS_FILENAME_CHAR_LENGTH)
        iterator = 0
        indexes_to_remove = []
        while iterator < self.validation_files:
            rand_index = random.randint(0, self.total_files-1)
            self.validation_data[iterator] = self.hotel_data[rand_index]
            indexes_to_remove.append(rand_index)
            iterator = iterator + 1
        
        # Training set:
        iterator = 0
        train_iterator = 0
        while iterator < self.total_files:
            if iterator in indexes_to_remove:
                iterator = iterator + 1
                continue
            else:
                if not train_iterator < self.validation_files:
                    break
                hotel_data_copy[train_iterator] = self.hotel_data[iterator]
                iterator = iterator + 1
                train_iterator = train_iterator + 1
        self.total_files = total_files_copy
        self.hotel_data = hotel_data_copy
        
        # Load test data:
        
        iterator = 0
        self.test_files=os.listdir(self.full_test_data_path)
        self.test_data = np.chararray(len(self.test_files), itemsize=MAX_WINDOWS_FILENAME_CHAR_LENGTH)
        print(self.test_data)
        for i,f in enumerate(self.test_files):
            self.test_data[i] = f
        print(self.test_data)
       
        
    def getValidationSet(self):
        return HotelDataset(DataSet=(self.validation_data, self.validation_files), transform=self.transform)
    def getTrainSet(self):
        return HotelDataset(DataSet=(self.training_data, self.training_files), transform=self.transform)
    def getTestSet(self):
        return HotelDataset(DataSet=(self.test_data, self.test_files), transform=self.transform)
            
    def __getitem__(self, index):
        label = self.hotel_data[index,1].decode()
        hotel_image_id = self.hotel_data[index,0].decode()
        image_path = os.path.join(self.full_data_path, label, hotel_image_id)
        
        image = Image.open(image_path)
        item = image
        if self.transform != None:
            item = self.transform(image)
        return item, label
        
    def __len__(self):
        return self.total_files

### Dataset class v2
##### used in rest of code

In [ ]:
class Hotel_Dataset_Merlijn(Dataset):
    """Dataset to load Hotel images."""
    
    def __init__(self, root_dir="../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/", data_path="train_images/",
                 labels_known = True, max_images=10, transform=None, label_list=None, save_names=False):
        # Setting variables:
        
        self.transform = transform
        self.max_images = max_images
        self.labels_known = labels_known
        
        # Directory paths:
        self.root_dir = root_dir
        self.data_path = data_path
        self.full_data_path = os.path.join(root_dir, data_path)
        dirs = os.listdir(self.full_data_path)

        # Initialise as empty dataset:
        data_size = NMBR_HOTELS*self.max_images
        if labels_known:
            self.data = np.chararray((data_size,2), itemsize=MAX_WINDOWS_FILENAME_CHAR_LENGTH)
            self.label_to_id_arr = np.zeros( NMBR_HOTELS, dtype=np.int32 )
        else:
            self.data = np.chararray(data_size, itemsize=MAX_WINDOWS_FILENAME_CHAR_LENGTH)
            if type(label_list) != None:
                self.label_to_id_arr = label_list
            else:
                print("Warning: no lable list provided for test dataset!")
            
        # Sampling Train or Val data (with labels known):
        
        if labels_known:
            for (dir_nmbr, this_dir) in enumerate(dirs):
                if dir_nmbr < NMBR_HOTELS:
                    self.label_to_id_arr[dir_nmbr] = int(this_dir)

                    # Sample pictures:
                    path_to_dir = os.path.join(self.full_data_path, this_dir)
                    files = os.listdir(path_to_dir)
                    sampled_pics = rnd.choice(files, size=self.max_images) #Note: these could be duplicates...

                    #Save as data:
                    data = np.dstack( (sampled_pics, np.repeat(dir_nmbr, self.max_images)))
                    (index_start, index_end) = ( (dir_nmbr)*self.max_images, (dir_nmbr+1)*self.max_images)
                    self.data[index_start: index_end] = data
            
        # Sampling Test data:
        
        else:
            self.data = os.listdir(self.full_data_path)
    
    def label_to_id(self, label):
        return self.label_to_id_arr[int(label)]
    
    def __len__(self):
        return len(self.data)
    
    def __size__(self):
        return np.shape(self.data)
    
    def __get_label_list__(self):
        return self.label_to_id_arr
    
    def __getitem__(self, idx):
        #note: has to be changed for no label types, still!
        
        if self.labels_known:
            record = self.data[idx].decode()
            label = int(record[1])
            hotel_id = self.label_to_id(label)
            image_path = os.path.join(self.full_data_path, str(hotel_id), record[0])
            image = pil_image.open(image_path)
            if self.transform:
                image = self.transform(image)
            
            return (image, label)
        
        else:
            record = self.data[idx]
            image_path = os.path.join(self.full_data_path, record)
            image = pil_image.open(image_path)

            if self.transform:
                image = self.transform(image)
            return (image, record)

In [ ]:
class Hotel_Dataset_Merlijn_Bram(Dataset):
    """Dataset to load Hotel images."""
    
    def __init__(self, root_dir="../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/", data_path="train_images/",
                 labels_known = True, images_per_hotel=20, ratio_in_training_partition=0.6, training=True, transform=None, shuffle=False):
        # Setting variables:
        max_images = self._n_images_in_partition(images_per_hotel, ratio_in_training_partition, training)
        
        MAX_WINDOWS_FILENAME_CHAR_LENGTH = 260 
        self.transform = transform
        self.max_images = max_images
        self.labels_known = labels_known
        
        # Directory paths:
        self.root_dir = root_dir
        self.data_path = data_path
        self.full_data_path = os.path.join(root_dir, data_path)
        dirs = os.listdir(self.full_data_path)

        # Initialise as empty dataset:
        data_size = NMBR_HOTELS*self.max_images
        if labels_known:
            self.data = np.chararray((data_size,2), itemsize=MAX_WINDOWS_FILENAME_CHAR_LENGTH)
            self.label_to_id_arr = np.zeros( NMBR_HOTELS, dtype=np.int32 )
        else:
            self.data = np.chararray(data_size, itemsize=MAX_WINDOWS_FILENAME_CHAR_LENGTH)
            
        # Sampling Train or Val data (with labels known):
        
        if labels_known:
            for (dir_nmbr, this_dir) in enumerate(dirs):
                if dir_nmbr < NMBR_HOTELS:
                    self.label_to_id_arr[dir_nmbr] = int(this_dir)

                    # Sample pictures:
                    path_to_dir = os.path.join(self.full_data_path, this_dir)
                    files = os.listdir(path_to_dir)

    #                 sampled_pics = rnd.choice(files, size=self.max_images) #Note: these could be duplicates...
    #                 sampled_pics = self._make_partition(files, training, self.max_images)
                    sampled_pics = self._make_partition2(files, training, images_per_hotel, ratio_in_training_partition)

                    #Save as data:
                    data = np.dstack( (sampled_pics, np.repeat(dir_nmbr, self.max_images)))
                    (index_start, index_end) = ( (dir_nmbr)*self.max_images, (dir_nmbr+1)*self.max_images)
                    self.data[index_start: index_end] = data
            
        # Sampling Test data:
        
        else:
            self.data = os.listdir(self.full_data_path)
    
    def _n_images_in_partition(self, images_per_hotel, ratio_in_first_partition, training):
        n_first_partition = int(np.ceil(images_per_hotel*ratio_in_first_partition))
        n_second_partition = images_per_hotel - n_first_partition
        return n_first_partition if training else n_second_partition
    
    def _repeatedly_append_to_reach_length(self, l, needed):
        if len(l) < 1:
            return []
        while needed > len(l):
            l.extend(l[:needed-len(l)])
        return l
    
#     def _make_partition(self, l: [str], make_first_partition = True, amount_per_partition = 10):
#         if not l:
#             return []
#         if len(l) == 1:
#             return l*amount_per_partition
#         taken_unique = l[:amount_per_partition*2]
#         if make_first_partition:
#             partition = l[:int(np.ceil(len(taken_unique)/2))]
#         else:
#             partition = l[int(np.ceil(len(taken_unique)/2)):amount_per_partition*2]
#         return self._repeatedly_append_to_reach_length(partition, amount_per_partition)
    
    def _make_partition2(self, l: [str], make_first_partition = True, images_per_hotel=20, ratio_in_first_partition=0.6):
        if not l:
            return []

        amount_in_first_partition = int(np.ceil(min(images_per_hotel, len(l))*ratio_in_first_partition))
        amount_in_second_partition = min(images_per_hotel, len(l)) - amount_in_first_partition
        expected_amount = amount_in_first_partition if make_first_partition else amount_in_second_partition
        expected_total_partition_1 = int(np.ceil(images_per_hotel*ratio_in_first_partition))
        expected_total_partition_2 = images_per_hotel - expected_total_partition_1
    #     expected_hotel = expected_total_partition_1 if make_first_partition else expected_total_partition_2

    #     if len(l) == 1:
    #         return l*expected_amount
    #     taken_unique = l[:images_per_hotel]
        if make_first_partition:
            partition = l[:max(1, amount_in_first_partition)]
            return self._repeatedly_append_to_reach_length(partition, expected_total_partition_1)
        else:
            partition = l[amount_in_first_partition:images_per_hotel]
            if len(partition) == 0:
                partition = l[:1]
            return self._repeatedly_append_to_reach_length(partition, expected_total_partition_2)
    
    def label_to_id(self, label):
        return self.label_to_id_arr[int(label)]
    
    def __len__(self):
        return len(self.data)
    
    def __size__(self):
        return np.shape(self.data)
    
    def __getitem__(self, idx, has_label = True):
        #note: has to be changed for no label types, still!
        
        if self.labels_known:
            record = self.data[idx].decode()
            label = int(record[1])
            hotel_id = self.label_to_id(label)
            image_path = os.path.join(self.full_data_path, str(hotel_id), record[0])
            image = pil_image.open(image_path)
            if self.transform:
                image = self.transform(image=image)
            
            return (image, label)
        
        else:
            record = self.data[idx]
            image_path = os.path.join(self.full_data_path, record)
            image = pil_image.open(image_path)

            if self.transform:
                image = self.transform(image)
            return (image, record)

### Defining data transformations:

In [ ]:
class AddRandomMaskTransform:
    
    def __init__(self):
        self.masks_path = r'../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_masks'
        self.max_covered_area_ratio=0.5
    
    def __call__(self, source_img: pil_image):
        import math
        source_width, source_height = source_img.size
        source_area = source_width*source_height
        
        # Retrieve a random mask
        random_mask_file_name = random.choice(os.listdir(self.masks_path))
        random_mask_path = os.path.join(self.masks_path, random_mask_file_name)
        random_mask = pil_image.open(random_mask_path)

        # The training masks provided by the competition often seem to be at least as large as some of the images
        # of the hotels. Resize the mask to constrain its surface area to max_covered_area_ratio, while retaining
        # its aspect ratio.
        original_mask_width, original_mask_height = random_mask.size
        original_mask_area = original_mask_width*original_mask_height
        max_mask_area = source_area*self.max_covered_area_ratio
        mask_width = math.floor(math.sqrt((max_mask_area*original_mask_width)/original_mask_height))
        mask_height = math.floor(math.sqrt((max_mask_area*original_mask_height)/original_mask_width))
        mask = random_mask.resize((mask_width, mask_height))

        # Paste the mask over a random position in the image
        max_x_coord = max(0, source_width - mask_width)
        max_y_coord = max(0, source_height - mask_height)
        random_x_coord = random.randint(0, max_x_coord)
        random_y_coord = random.randint(0, max_y_coord)
        source_img.paste(mask, (random_x_coord, random_y_coord), mask)
        # Mind that most masks have a transparent border that offsets them from (0,0), preventing
        # most of them from ever showing up all the way at the left or the top of the image.
        return source_img

In [ ]:
IMAGE_SIZE_WIDTH = 256
IMAGE_SIZE_HEIGHT = 256

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomChoice([
        torchvision.transforms.Resize((IMAGE_SIZE_WIDTH, IMAGE_SIZE_HEIGHT)),
        torchvision.transforms.RandomCrop((IMAGE_SIZE_WIDTH, IMAGE_SIZE_HEIGHT))
    ]),
    AddRandomMaskTransform(),
#     torchvision.transforms.Resize((IMAGE_SIZE_WIDTH, IMAGE_SIZE_HEIGHT)),
    torchvision.transforms.RandomApply(torch.nn.ModuleList([torchvision.transforms.RandomRotation((0, 359))]),p=0.3),
    torchvision.transforms.RandomApply(torch.nn.ModuleList([torchvision.transforms.RandomHorizontalFlip()]),p=0.3),
    torchvision.transforms.RandomApply(torch.nn.ModuleList([torchvision.transforms.RandomVerticalFlip()]),p=0.3),
    #torchvision.transforms.Grayscale(),
    torchvision.transforms.ToTensor()
]) #This one should definetly be expanded!

test_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomChoice([
        torchvision.transforms.Resize((IMAGE_SIZE_WIDTH, IMAGE_SIZE_HEIGHT)),
        torchvision.transforms.RandomCrop((IMAGE_SIZE_WIDTH, IMAGE_SIZE_HEIGHT))
    ]),
    AddRandomMaskTransform(),
#     torchvision.transforms.Resize((IMAGE_SIZE_WIDTH, IMAGE_SIZE_HEIGHT)),
    torchvision.transforms.RandomApply(torch.nn.ModuleList([torchvision.transforms.RandomRotation((0, 359))]),p=0.3),
    torchvision.transforms.RandomApply(torch.nn.ModuleList([torchvision.transforms.RandomHorizontalFlip()]),p=0.3),
    torchvision.transforms.RandomApply(torch.nn.ModuleList([torchvision.transforms.RandomVerticalFlip()]),p=0.3),
    #torchvision.transforms.Grayscale(),
    torchvision.transforms.ToTensor()
])

### Creating Datasets & Loaders

In [ ]:
# Method 1: both sets have each hotel represented, but duplicates are possible:

#dataset_train = Hotel_Dataset_Merlijn(max_images = 1, transform=train_transform)
#dataset_val = Hotel_Dataset_Merlijn(max_images = 1, transform=train_transform)
# Not sure if we cannot just use the same one twice, but never mind...


# Method 2: only duplicates in dataset may occur, no guarantee about labels
nmbr_images, val_perc = 20, 0.9
dataset_total = Hotel_Dataset_Merlijn(max_images = nmbr_images, transform=train_transform)
train_len = m.ceil(nmbr_images*val_perc*NMBR_HOTELS)
val_len = nmbr_images*NMBR_HOTELS - train_len
dataset_train, dataset_val = torch.utils.data.random_split(dataset_total, [train_len, val_len])

labels = dataset_total.__get_label_list__()
dataset_test = Hotel_Dataset_Merlijn(data_path="test_images/", transform=train_transform, labels_known = False, label_list = labels)
     
train_loader = DataLoader(dataset_train, batch_size = 25, shuffle=True)
val_loader = DataLoader(dataset_val, batch_size = 25, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size = 1) #this seems sloppy, but makes predict-code nicer now...

# Method 3
# dataset_train = Hotel_Dataset_Merlijn_Bram(images_per_hotel=10, ratio_in_training_partition=0.6, training=True, transform=train_transform)
# dataset_val = Hotel_Dataset_Merlijn_Bram(images_per_hotel=10, ratio_in_training_partition=0.6, training=False, transform=test_transform)
# dataset_test = Hotel_Dataset_Merlijn_Bram(labels_known=False, data_path="test_images/", images_per_hotel=10, ratio_in_training_partition=0.6, training=False, transform=test_transform)
# dataset_total = Hotel_Dataset_Merlijn_Bram(images_per_hotel=10, ratio_in_training_partition=0.6, training=False, transform=test_transform)

# train_loader = DataLoader(dataset_train, batch_size = 5, shuffle=True)#len(dataset_train))
# val_loader = DataLoader(dataset_val, batch_size = 5, shuffle=True)
# test_loader = DataLoader(dataset_test, batch_size = 1, shuffle=False)



### Visualising/testing data stuff

In [ ]:
# Test/visualise datasets:

# Function to visualise datasets:
def show_images(ds, n_images=10, labels_known = True):
    # Only works properly for n_images>10...
    xlen, ylen = 10, m.ceil(n_images/10)
    fig, ax = plt.subplots(ylen,xlen, figsize=(22,8))
    
    
    for x in range(xlen):
        for y in range(ylen):
            if not labels_known:
                item = ds.__getitem__(y*10+x)
            else:
                (item, label) = ds.__getitem__(y*10+x)
            ax[y,x].imshow(np.transpose(item, (1,2,0))) # gives error: convert to PIL

def visualise_data():
    print("""Sizes Datasets:
        Train: {0}
        Validate: {1}
        Test: {2}""")#.format(dataset_train.__size__(), dataset_val.__size__(), dataset_test.__size__()))
    print(dataset_train[0])

    show_images(dataset_train,20, labels_known = True) # This gives an error, but I am too lazy to fix it now...
#visualise_data()



## Auxiliariy functions

In [ ]:
def save_checkpoint(model, scheduler, optimizer, epoch, name, loss=None, score=None):
    checkpoint = {"epoch": epoch,
                  "model": model.state_dict(),
                  #"scheduler": scheduler.state_dict(),
                  "optimizer": optimizer.state_dict(),
                  "loss": loss,
                  "score": score,
                  }

    torch.save(checkpoint, f"{OUTPUT_FOLDER}checkpoint-{name}.pt")



## Model

In [ ]:
# Define model:

class Hotel_Model (nn.Module):
    
    def __init__(self, n_classes = NMBR_HOTELS, embedding_size = NMBR_EMBEDDINGS):
        super(Hotel_Model, self).__init__()
        
        # use (soon-to-be) pre-trained model for heavy lifting:
        self.main_model = timm.create_model("efficientnet_b0", num_classes=NMBR_HOTELS, pretrained=False)
        #self.main_model.load_state_dict(torch.load("../input/timm-pretrained-efficientnet/efficientnet/efficientnet_b0_ra-3dd342df.pth"))
        #self.main_model = models.alexnet()
        
        #for param in self.main_model.parameters():
        #    param.requires_grad = False
        
        self.main_model.eval()
        
        
        # Use own torch-layers for last embedding step and classification:
        #in_features = self.main_model.get_classifier().in_features
        #in_features = self.main_model.classifier[1].in_features      
        #self.main_model.classifier = nn.Identity() # turn off classifier in pre-trained model
        #in_features = 100
        #self.main_model = nn.Linear(1000, in_features)
        
        #self.embedding = nn.Sequential(
        #    nn.ReLU(),
        #    nn.Linear(1000, embedding_size),
        #    nn.ReLU())
        
        #Add own final layer and classifier !!! CHANGE THIS TO RETRIEVAL !!!
        #self.classifier = nn.Sequential(
        #    nn.Linear(embedding_size, n_classes),
        #    nn.Softmax(dim=1))
        
        self.classifier = nn.Softmax(dim=1)
    def forward(self,x):
        x = self.main_model(x)
        #print(x)
        #x = x.view(x.size(0), -1) #re-scale
        #x = self.embedding(x)
        #print(x)
        return x,self.classifier(x) #self.classifier(x)
    
model_test = Hotel_Model()

## Training Cycle

In [ ]:
def train_epoch(train_loader, optimizer, loss_fn, model):
    """Training and adjusting model for one epoch."""
    
    for i, data in enumerate(train_loader): #loop over all data in current batch (in loader!)        
        # Initialisation:
        optimizer.zero_grad()
        inputs, labels =  data
        #print(inputs)
        
        # Make prediction, compute loss
        inputs = inputs.to(device)
        labels = labels.to(device)
        embedding, prediction = model(inputs)
        loss = loss_fn(prediction, labels)
        loss.backward()
        
        # Adjust model 
        optimizer.step()
        
        embedding = embedding.to("cpu")
        prediction = prediction.to("cpu")
        labels = labels.to("cpu")
        
        pi, l = prediction[0], labels[0]
        
        # Log progress
        prediction, labels = prediction.detach().numpy(), labels.detach().numpy()
        #print(prediction[0], labels[0])
        accuracy = np.mean(labels == np.argmax(prediction, axis = 1))
        
        print("Training batch {} done! (accuracy = {}, Loss = {}, label = {})".format(i+1, accuracy, loss, labels[0]))
        
        
        
        embsf, ps = model(inputs)
        pf = ps[0]
        #print((pi,pf,l))
        
        
        
        # Gather data and stuff: TBW
    
    print("All training completed!")
    
    # Clear all arrays from memory:
    data, inputs, labels, embedding, prediction, loss = [],[],[],[],[],[]
    
def val_epoch(val_loader, loss_fn, model):
    "Compute validation data for one epoch"
    
    # Just like train_epoch, but without adjusting the model...
    total_vloss = 0.0
    avg_accuracy = 0.0
    nmbr_images = 0.0
    
    for i, vdata in enumerate(val_loader):
        nmbr_images += len(vdata)
        inputs, labels = vdata
        inputs = inputs.to(device)
        labels = labels.to(device)
        emb, prediction = model(inputs)
        vloss = loss_fn(prediction, labels)
        total_vloss += vloss
        top_pred = np.argmax(prediction, axis=1)
        accuracy = np.mean((top_pred == labels).detach().numpy().astype(int))
        avg_accuracy += accuracy
        print( "Validation batch {0} done! Accuracy: {1}".format((i+1), accuracy) )
    
    vdata, inputs, labels, emb, prediction = [],[],[],[],[]
    return (total_vloss / (i+1), avg_accuracy / (i+1) )

In [ ]:
def train_complete(train_loader, val_loader, model, optimizer, scheduler, loss_fn, nmbr_epochs = 5):
    """Trains and Validates model given loaders and initial model."""

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    best_vloss = 10**9
    for epoch in range(nmbr_epochs):
        print("Starting epoch {}".format(epoch+1))
        
        # Train one epoch:
        model.train(True)
        train_epoch(train_loader, optimizer, loss_fn, model)
        scheduler.step()
        
        # Validate epoch:
        print("Start Validation...")
        model.train(False)
        with torch.no_grad():
            (vloss, acc) = val_epoch(val_loader,loss_fn, model)
        print("Validation complete: Loss = {}, Accuracy = {}! Save model & start new epoch:".format(vloss, acc))
        
        # Save current model:
        #save_checkpoint(model, "test", optimizer, epoch, "TestTraining")     

import numpy as np
#import faiss

a1 = np.array([0.,0.,0.,0.,0.])
a2 = np.array([1.,2.,3.,4.,4.])
a3 = np.array([1.,2.,3.,3.,3.])
a4 = np.array([1.,2.,2.,2.,2.])
embeddings = np.array([a3,a1,a4,a2])
embeddings = embeddings.astype(np.float32)
print("Embeddings: \n{}".format(embeddings))

target = np.array([1.,2.,3.,4.,5.])
target = target.astype(np.float32)
print("Target: \n{}".format(target))

n = 3

def find_nearest_n_neighbours(target, embeddings, n: int):
    index =faiss.IndexFlatL2(len(target))
    index.add(embeddings)
    
    distances, indices = index.search(np.array([target]), n)
    print("Indices: \n{}".format(indices))
    print("Distances: \n{}".format(distances))
    
find_nearest_n_neighbours(target,embeddings,n)

In [ ]:
def predict(test_loader, model, label_to_id):
    """Function to predict the 5 most likely hotels"""
    
    nmbr_cases = len(test_loader)
    predictions_top_5 = np.chararray((nmbr_cases ,2), itemsize=MAX_WINDOWS_FILENAME_CHAR_LENGTH)
    
    for i,data in enumerate(test_loader): #Should just be one batch...
        (image, name) = data
        predictions_top_5[i,0] = name[0] # Don't know why we need the [0]...
        image = image.to(device)
        emb, prediction = model(image)
        prediction = prediction.to("cpu")
        pred_np = prediction.detach().numpy()[0]
        
        pred_string = ""
        for j in range(5):
            ind = np.argmax(pred_np)
            pred_np[ind] = -10
            pred_string += str(label_to_id(ind))+" "
        predictions_top_5[i,1] = pred_string
    return predictions_top_5

def predict_new(test_loader, model):
    embeddings = model.embed()

## Running the code & Making submission:

In [ ]:
CUDA_LAUNCH_BLOCKING=1

In [ ]:
device = ('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
# Define model & Datasets
model = Hotel_Model()

# Move model to GPU if available
model = model.to(device)

# Optimizer & Loss function: Placeholders for now:

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.5, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=1)
loss_fn = torch.nn.CrossEntropyLoss()
#loss_fn = torch.nn.NLLLoss()

In [ ]:
# Train model:

nmbr_epochs = 25
train_complete(train_loader, val_loader, model, optimizer, scheduler, loss_fn, nmbr_epochs)

# Make prediction:

final_preds = predict(test_loader, model, dataset_total.label_to_id ) # predictions as np-array
final_preds = pd.DataFrame(final_preds).stack().str.decode('utf-8').unstack() # as panda dataFrame of strings
final_preds.set_axis({'image_id','hotel_id',}, axis=1, inplace=True)  # rename header

print(final_preds)
final_preds.to_csv(OUTPUT_FOLDER + 'submission.csv', index=False)